In [3]:
import torch
import numpy as np

## tensor 생성 및 변환

In [2]:
print(torch.tensor([[1,2],[3,4]]))
print(torch.tensor([[1,2],[3,4]], device='cuda:0')) # GPU에 텐서 생성
print(torch.tensor([[1,2],[3,4]], dtype=torch.float64)) # dtype으로 텐서 타입 정할 수 있음

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='cuda:0')
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


### tensor to ndarray

In [9]:
temp = torch.tensor([[1,2],[3,4]])
print(temp.numpy(), temp.numpy().dtype, type(temp.numpy()))

[[1 2]
 [3 4]] int64 <class 'numpy.ndarray'>


In [8]:
# gpu 할당 된 텐서는 cpu로 바꾸고나서 ndarray로 변환해준다.
temp = torch.tensor([[1,2],[3,4]], device='cuda:0')
print(temp.to('cpu').numpy()) # GPU 상의 텐서를 CPU의 텐서로 변환 후 ndarray로 변환

numpy.ndarray

## tensor index manipulation

### tensor를 지정 타입으로 만드는 함수
- torch.FloatTensor : 32비트의 부동 소수점
- torch.DoubleTensor : 64비트의 부동 소수점
- torch.LongTensor : 64비트의 부호가 있는 정수

In [12]:
temp = torch.FloatTensor([1,2,3,4,5,6,7]) # 1차원 벡터 생성
print(temp[0], temp[1], temp[-1]) # 인덱스 접근
print(temp[2:5], temp[3:-2])

tensor(1.) tensor(2.) tensor(7.)
tensor([3., 4., 5.]) tensor([4., 5.])


### tensor 연산 및 차원 조작
- 텐서 간 타입이 다르면 연산이 불가

In [13]:
v = torch.tensor([1,2,3])
w = torch.tensor([3,4,5])
print(w-v)

tensor([2, 2, 2])


- 텐서 차원 변경 대표 방법은 view를 사용하는 것.
- stack, cat
- t, transpose
---
- view : 넘파이의 reshape와 비스스
- cat : 다른 길이의 텐서를 하나로 병합할 떄 사용

In [14]:
temp = torch.tensor([[1,2],[3,4]])

In [16]:
print(temp.shape)
print('-------------------------')
print(temp.view(4,1)) # 2x2 -> 4x1 로 변현
print('-------------------------')
print(temp.view(-1)) # 2x2 -> 1차원 벡터로 변형
print('-------------------------')
print(temp.view(1,-1)) # -1은 (1,?)와 같은 의미로 다른 차원으로부터 해당 값을 유추하겠다는 의미.
print('-------------------------')
print(temp.view(-1,1))

torch.Size([2, 2])
-------------------------
tensor([[1],
        [2],
        [3],
        [4]])
-------------------------
tensor([1, 2, 3, 4])
-------------------------
tensor([[1, 2, 3, 4]])
-------------------------
tensor([[1],
        [2],
        [3],
        [4]])


## data prepare
- 데이터가 이미지일 경우 분산된 파일에서 데이터를 읽은 후 전처리를 하고 배치 단위로 분할하여 처리한다.
- 데이터가 텍스트일 경우 임베딩 과정을 거쳐 서로 다른 길이의 시퀀스를 배치 단위로 분할하여 처리한다.


### DataLoader
- 학습에 사용될 데이터 전체를 보관했다가 모델 학습을 할 때 배치 크기만큼 데이터를 꺼내서 사용한다.
- 미리 데이터를 배치로 잘라 놓는 것이 아니라, 내부적으로 iterator가 index를 사용해 배치크기만큼 필요할 때 반환해주는 것

### 파이토치 제공 데이터
- torchvision : 파이토치에서 제공하는 데이터셋들이 모여 있는 패키지
- https://pytorch.org/vision/0.8/datasets.html
- 파이토치 제공 데이터를 받으려면 requests 라이브러리 필요

In [17]:
import torchvision.transforms as transforms

In [19]:
mnist_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,), (1.0,)) # 평균0.5, 표준편차1.0 되도록 normalize
])

In [20]:
from torchvision.datasets import MNIST
import requests

download_root = '.'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## Model define
- 계층(layer) : 모듈 또는 모듈을 구성하는 한 개의 계층으로 합성곱층, 선형계층 등이 있다.
- 모듈(module) : 한 개 이상의 계층이 모여서 구성된 것으로, 모듈이 모여 새로운 모듈을 만들 수도 있다.
- 모델(model) : 최종적으로 원하는 네트워크로, 한 개의 모듈이 모델이 될 수도 있다.



### 단순 신경망 정의
- nn.Module은 상속받지 않는 매우 단순한 모델을 만든다.

### nn.Module() 상속 정의
- `__init__()`, `forward()`함수를 포함
- `__init__()` : 레이어 설정
- `forward()` : 계산 정의 

### Sequential 신경망 정의
- nn.Sequential 사용하면 가독성이 좋다.

In [22]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))

        self.layer3 = nn.Sequential(
            nn.Linear(in_features=30*5*5, out_features=10, bias=True),
            nn.ReLU(inplace=True))
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.layer3(x)
        return x

In [23]:
model = MLP()

In [24]:
print('Printing children')
print(list(model.children()))

print('Printing Modules')
print(list(model.modules()))

Printing children
[Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=750, out_features=10, bias=True)
  (1): ReLU(inplace=True)
)]
Printing Modules
[MLP(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=750, out_features=10, bias=True)
    (1): ReLU(inp